<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-16 11:45:51
-------------------
qualified stocks: 96
with latest results: 26
still star stocks: 16


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  70.78 K
Current:  1.28 C
-------------------
Today PnL: 95.78 K (0.76%)
Current PnL: -16.00 L (-11.35%)
CY Booked + Current PnL: -13.13 L (-9.31%)
-------------------
Total profit:  4.38 L
Total loss:  -20.38 L
-------------------
Total Booked + Current PnL: 13.85 L (12.14%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 79.07 L (61.95%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (110.12%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.13%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,MRF,114777.60,150504.12,15.30,X-MC,78.12,141300.0,26522.0,9199.0,0.11,23.11,6.51,31.13,220.0,2.88,1.11,37.64,AR,ATH,TYRES
63,RELIANCE,1288.53,1526.00,6.50,X-LC,93.23,229495.0,25907.0,11612.0,-0.12,12.73,5.06,18.43,157.0,2.23,1.81,25.01,XY25,NTT,REFINERIES
70,SBILIFE,1496.49,1924.99,0.61,H-LC,80.21,149520.0,23815.0,12186.0,-0.26,18.94,8.15,28.63,147.0,1.95,1.18,33.20,AR,ATH,INSURANCE
57,PIDILITIND,2504.06,3576.00,-16.01,X-LC,34.38,93114.0,17992.0,14163.0,-0.25,23.95,15.21,42.81,120.0,1.27,0.73,16.78,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-14.39,M-LC,41.67,210238.0,8758.0,18459.0,0.19,4.35,8.78,13.51,60.0,0.47,1.66,17.49,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TITAGARH,1097.23,1548.0,-10.32,H-SC,82.29,157588.0,-34427.0,113306.0,11.30,-17.93,71.90,41.08,95.0,-0.30,1.24,33.38,XY24,NTT,ENGINEERING
90,VALIANTORG,512.64,838.0,-325.20,H-SC,48.96,95628.0,-37658.0,122251.0,7.00,-28.25,127.84,63.47,161.0,-0.31,0.75,60.30,XR,NTT,CHEMICALS
6,ANGELONE,2328.67,3033.0,9.21,H-SC,87.50,177678.0,26314.0,19474.0,6.42,17.38,10.96,30.25,111.0,1.35,1.40,40.63,X40N,NTT,FINANCE
92,VIPIND,488.80,718.0,-880.60,H-SC,100.00,74354.0,-20962.0,65655.0,4.24,-21.99,88.30,46.89,82.0,-0.32,0.59,46.65,X40N,NTT,MISC
42,IRCTC,904.87,1269.0,-23.33,M-MC,53.12,126360.0,-14800.0,71608.0,3.09,-10.48,56.67,40.24,30.0,-0.21,1.00,20.72,XY24,BTT,TRAVEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1022.60,1486.00,2.85,X-MC,2.08,181154.0,25719.0,44727.0,-4.14,16.55,24.69,45.32,201.0,0.58,1.43,23.26,XY24,NTT,HEALTHCARE
40,INDUSINDBK,1354.50,1800.00,-41.30,H-MC,6.25,43334.0,-33872.0,59264.0,-2.60,-43.87,136.76,32.89,24.0,-0.57,0.34,19.34,XR,NTT,BANKS
41,INFY,1432.81,2275.00,-11.34,X-LC,73.96,220649.0,21488.0,95585.0,-1.59,10.79,43.32,58.78,10.0,0.22,1.74,15.20,X40,BTT,IT
79,SYMPHONY,1311.84,2013.49,-20.64,M-SC,75.00,148430.0,-2432.0,83121.0,-1.38,-1.61,56.00,53.49,15.0,-0.03,1.17,30.61,X40N,ATH,DURABLES
78,SURYODAY,189.78,240.00,40.35,H-SC,38.02,99445.0,-50481.0,90157.0,-1.32,-33.67,90.66,26.46,88.0,-0.56,0.78,27.40,XR,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TATAMOTORS,776.02,1065.00,-51.41,M-LC,95.83,194669.0,-10976.0,87562.0,0.89,-5.34,44.98,37.24,1.0,-0.13,1.53,26.71,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-52.03,H-SC,70.83,103528.0,1801.0,112576.0,0.96,1.77,108.74,112.43,2.0,0.02,0.82,22.19,AR,ATH,MISC
2,ABB,5319.37,8953.12,-36.66,H-LC,62.50,145000.0,12016.0,78822.0,1.13,9.04,54.36,68.31,4.0,0.15,1.14,18.24,AR,ATH,ELECTRICAL
83,TCS,3928.29,4998.00,-14.87,X-LC,63.54,191916.0,-20212.0,77975.0,-0.72,-9.53,40.63,27.23,5.0,-0.26,1.51,9.98,X40,BTT,IT
49,LICI,777.74,983.00,-13.28,H-LC,57.29,192808.0,18594.0,27379.0,2.43,10.67,14.20,26.39,6.0,0.68,1.52,18.04,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,MRF,114777.60,150504.12,15.30,X-MC,78.12,141300.0,26522.0,9199.0,0.11,23.11,6.51,31.13,220.0,2.88,1.11,37.64,AR,ATH,TYRES
57,PIDILITIND,2504.06,3576.00,-16.01,X-LC,34.38,93114.0,17992.0,14163.0,-0.25,23.95,15.21,42.81,120.0,1.27,0.73,16.78,X40,BTT,CHEMICALS
58,POLYCAB,5005.92,7541.35,8.65,H-MC,85.42,169932.0,29766.0,41226.0,0.45,21.24,24.26,50.65,55.0,0.72,1.34,29.69,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,83.33,192896.0,41690.0,23977.0,0.13,27.57,12.43,43.43,37.0,1.74,1.52,33.71,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,35.79,M-SC,36.46,141066.0,3488.0,90931.0,2.06,2.54,64.46,68.62,79.0,0.04,1.11,22.29,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,9.21,H-SC,87.50,177678.0,26314.0,19474.0,6.42,17.38,10.96,30.25,111.0,1.35,1.40,40.63,X40N,NTT,FINANCE
58,POLYCAB,5005.92,7541.35,8.65,H-MC,85.42,169932.0,29766.0,41226.0,0.45,21.24,24.26,50.65,55.0,0.72,1.34,29.69,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,83.33,192896.0,41690.0,23977.0,0.13,27.57,12.43,43.43,37.0,1.74,1.52,33.71,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-1.91,H-SC,66.67,144687.0,6827.0,30934.0,2.96,4.95,21.38,27.39,68.0,0.22,1.14,26.65,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-4.44,H-SC,14.58,107377.0,5373.0,39654.0,2.91,5.27,36.93,44.14,22.0,0.14,0.85,22.36,XR,ATH,MISC
95,WIPRO,243.46,420.00,-7.89,M-LC,33.33,157517.0,6572.0,102890.0,-0.98,4.35,65.32,72.51,52.0,0.06,1.24,18.60,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-33.09,M-MC,16.67,130506.0,1354.0,85716.0,0.55,1.05,65.68,67.42,41.0,0.02,1.03,26.93,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-52.03,H-SC,70.83,103528.0,1801.0,112576.0,0.96,1.77,108.74,112.43,2.0,0.02,0.82,22.19,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,79.17,150500.0,-413.0,60862.0,-0.58,-0.27,40.44,40.06,14.0,-0.01,1.19,23.10,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-325.20,H-SC,48.96,95628.0,-37658.0,122251.0,7.00,-28.25,127.84,63.47,161.0,-0.31,0.75,60.30,XR,NTT,CHEMICALS
87,UJJIVANSFB,52.77,60.00,40.02,M-SC,84.38,121500.0,-20979.0,40496.0,-0.90,-14.72,33.33,13.70,214.0,-0.52,0.96,43.40,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,16.67,130506.0,1354.0,85716.0,0.55,1.05,65.68,67.42,41.0,0.02,1.03,26.93,XR,NTT,BANKS
54,MRF,114777.60,150504.12,15.30,X-MC,78.12,141300.0,26522.0,9199.0,0.11,23.11,6.51,31.13,220.0,2.88,1.11,37.64,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,0.61,H-LC,80.21,149520.0,23815.0,12186.0,-0.26,18.94,8.15,28.63,147.0,1.95,1.18,33.20,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-880.60,H-SC,100.00,74354.0,-20962.0,65655.0,4.24,-21.99,88.30,46.89,82.0,-0.32,0.59,46.65,X40N,NTT,MISC
81,TATAELXSI,7332.28,9000.00,-17.32,X-MC,98.96,86849.0,-15803.0,39152.0,0.37,-15.39,45.08,22.74,47.0,-0.40,0.68,30.54,X40N,NTT,IT
80,TANLA,917.30,1963.11,-39.00,H-SC,97.92,149657.0,-89758.0,362709.0,3.05,-37.49,242.36,114.01,20.0,-0.25,1.18,37.26,AR,ATH,IT
76,SONACOMS,578.05,1006.00,-26.97,M-MC,96.88,94369.0,-6790.0,81686.0,1.03,-6.71,86.56,74.03,49.0,-0.08,0.74,32.62,AR,BTT,AUTO
82,TATAMOTORS,776.02,1065.00,-51.41,M-LC,95.83,194669.0,-10976.0,87562.0,0.89,-5.34,44.98,37.24,1.0,-0.13,1.53,26.71,XY24,NTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.14
1,25,38.57
2,50,67.60


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.63
X40N    15.23
AR      15.10
X40     14.24
XR      10.27
XY25     8.28
X200     1.19
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.44
M-SC    17.69
X-LC    16.08
H-MC     8.93
M-LC     8.31
X-MC     6.65
H-LC     5.90
M-MC     5.54
X-SC     2.80
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.61,-0.66,32.68
BANKS,10.89,-9.04,62.53
IT,9.15,-19.10,85.54
FINANCE,7.44,-17.31,70.25
MISC,7.16,-24.92,78.68
INSURANCE,6.11,0.38,29.65
PAINTS,5.56,-16.62,51.13
AUTO,4.37,-9.00,56.51
ELECTRICAL,4.10,-6.68,61.04


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2957825.0,27
AR,1298596.0,16
XR,1242243.0,15
X40N,1231351.0,18
X40,787146.0,12
SR,203968.0,2
XY25,119279.0,5
X200,66383.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2341080.0,22
M-SC,1993810.0,21
X-LC,699847.0,12
M-MC,566106.0,5
H-MC,475658.0,8
X-SC,353432.0,4
M-LC,326126.0,6
L-SC,311402.0,4
X-MC,310120.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      897191.0      6
M-SC       XY24      749961.0      6
H-SC       AR        535088.0      3
           X40N      528147.0      7
M-SC       X40N      448219.0      6
           XR        423742.0      5
M-MC       XY24      398704.0      3
X-LC       X40       376699.0      5
H-SC       XR        291362.0      4
M-LC       XY24      204777.0      4
H-LC       AR        196813.0      4
M-SC       X40       194812.0      2
X-LC       AR        183324.0      3
H-MC       XY24      169981.0      2
L-SC       XR        161820.0      2
X-MC       XY24      146998.0      2
H-MC       AR        134878.0      2
X-SC       SR        120113.0      1
           XR        117449.0      1
           X40N      115870.0      2
M-LC       XR        102890.0      1
X-MC       X40N       97889.0      2
M-SC       AR         93221.0      1
H-SC       X40        89292.0      2
M-MC       XR         85716.0      1
L-SC       XY24       85195.0      1
M-SC       SR         83855.0      1
M-MC       AR         81686.0      1
X-LC       XY25       73441.0      3
H-MC       X40        70309.0      2
X-LC       X200       66383.0      1
L-SC       AR         64387.0      1
H-MC       XR         59264.0      1
X-MC       X40        56034.0      1
H-MC       X40N       41226.0      1
H-LC       XY25       27379.0      1
M-LC       XY25       18459.0      1
X-MC       AR          9199.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
